## **Train the model on the Google AI Platform**

In [1]:
## Mount on google drive
from google.colab import drive
drive.mount('/content/drive/')
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()
## to the work directorty
import os
os.chdir("/content/drive/My Drive/Earth-Engine-with-Deep-Learning/trainer")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#### <font color=red> **！！Note: the runtime-version setting (supported tensorflow version in AI platform) should be consistency with the Tensorflow version in the colab, and we use Tensorflow 2.2.0 here!**

In [ ]:
# !pip install tensorflow==2.2.0
# import tensorflow as tf
# print(tf.__version__)

In [ ]:
import time
from ai_platform_package import config
from ai_platform_package import model
from ai_platform_package import dataLoader

Job_Name = 'impervious_unet_training_job_' + time.strftime("%Y%m%d%H%M")
Train_Package_Path = 'ai_platform_package'
Main_Trainer_Module = 'ai_platform_package.trainingTask'
Region = 'asia-east1'


## Check the configuration and training modules

In [ ]:
## Check the configuration file.
# !cat ai_platform_package/config.py
trainData = dataLoader.get_training_dataset()
# print(iter(trainData.take(1)).next())
model = model.UNet(input_shape=(256, 256, 6), nclasses=2)
# print(model.summary())


## Submit the package to AI Platform for training

In [ ]:
!gcloud ai-platform jobs submit training {Job_Name} \
    --job-dir {config.Job_Dir} \
    --package-path {Train_Package_Path} \
    --module-name {Main_Trainer_Module} \
    --region {Region} \
    --project {config.Project} \
    --runtime-version 2.2 \
    --python-version 3.7 \
    --scale-tier basic-gpu


In [ ]:
desc = !gcloud ai-platform jobs describe {Job_Name} --project {config.Project}
state = desc.grep('state:')[0].split(':')[1].strip()
print(state)


## Inspect the trained model

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {config.Logs_Dir}
